# Flights data exploration
# Wrangling
## by Kacper Fiszer
I am going to look into a dataset encompassing United States flight data in the years 1987-2008.  
The data includes detailed flight information, including delay times and reasons, and comes in the form of csv files.  
First, I will explore the files, see if all the data is relevant and pose further exploration questions.  
  
Column descriptions are available here:  
[http://stat-computing.org/dataexpo/2009/the-data.html](http://stat-computing.org/dataexpo/2009/the-data.html)

In [1]:
# import needed packages
import numpy as np
import pandas as pd

## Gather
The data was supposed to be available under the above link, but wasn't.  
Downloaded the same file from: [https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7)

Extracted files are in the csv format, year by year. I put them all in `data` directory.

## Assess
### Part I - multiple files at a glance

**Assessing basic file integrity**  
Files with 1987 and 2008 data seem much smaller than all other years in range: 100-200 MB compared to usual 400-600 MB.  
This most probably signifies incomplete data.

In [2]:
df_1987 = pd.read_csv('./data/1987.csv')
df_1987.shape

(1311826, 29)

In [3]:
df_2007 = pd.read_csv('./data/2007.csv')
df_2007.shape

(7453215, 29)

In [4]:
df_2008 = pd.read_csv('./data/2008.csv')
df_2008.shape

(2389217, 29)

Row counts show that there is indeed much fewer records for 1987 and 2008.  
Just to confirm, I'll check if this has to do with data being cut off at specific points.

In [5]:
df_1987.Month.value_counts().sort_index()

10    448620
11    422803
12    440403
Name: Month, dtype: int64

In [6]:
df_2007.Month.value_counts().sort_index()

1     621559
2     565604
3     639209
4     614648
5     631609
6     629280
7     648560
8     653279
9     600187
10    629992
11    605149
12    614139
Name: Month, dtype: int64

In [7]:
df_2008.Month.value_counts().sort_index()

1    605765
2    569236
3    616090
4    598126
Name: Month, dtype: int64

It is confirmed that 1987 and 2008 data is incomplete - only contains records for part of a year.  
**For the sake of this project I will pick a 3-year period of 2005-2007**

In [8]:
# read in csv files by year to separate variables
for y in range(2005, 2007+1) :
    globals()['df_%s' %y] = pd.read_csv('./data/'+str(y)+'.csv')

In [9]:
# test if all csv's have the same column names (output should be 1)
test = list()
for y in range(2005, 2007+1) :
    test.append(globals().get('df_%s' %y).columns == df_2005.columns)
np.array(test).mean()

1.0

In [10]:
# test if columns in each df have the same data type (output should be 1)
test = list()
for y in range(2005, 2007+1) :
    test.append(globals().get('df_%s' %y).dtypes == df_2005.dtypes)
np.array(test).mean()

0.9770114942528736

**Some columns have different data types across yearly csv's**  
Let's find the differing columns.

In [11]:
# Loop through column dtypes for each year and print those different to 2005
diff_list = list()

for y in range(2005, 2007+1) :
    for col in range(0, 29) :
        if globals().get('df_%s' %y).dtypes[col] != df_2005.dtypes[col] :
            diff_list.append(str(y) + ' ' + str(globals().get('df_%s' %y).columns[col])
                             + ' ' + str(globals().get('df_%s' %y).dtypes[col]))

diff_list

['2006 CRSElapsedTime float64', '2007 CRSElapsedTime float64']

These above columns should be integers rather than floats, as they refer to 'scheduled elapsed time in minutes'.  
Let's look at the summary for one of the sets.

In [12]:
df_2006.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7141922 entries, 0 to 7141921
Data columns (total 29 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Year               7141922 non-null  int64  
 1   Month              7141922 non-null  int64  
 2   DayofMonth         7141922 non-null  int64  
 3   DayOfWeek          7141922 non-null  int64  
 4   DepTime            7019988 non-null  float64
 5   CRSDepTime         7141922 non-null  int64  
 6   ArrTime            7003802 non-null  float64
 7   CRSArrTime         7141922 non-null  int64  
 8   UniqueCarrier      7141922 non-null  object 
 9   FlightNum          7141922 non-null  int64  
 10  TailNum            7141922 non-null  object 
 11  ActualElapsedTime  7003802 non-null  float64
 12  CRSElapsedTime     7141918 non-null  float64
 13  AirTime            7003802 non-null  float64
 14  ArrDelay           7003802 non-null  float64
 15  DepDelay           7019988 non-n

Most of the columns had been treated as integer, therefore the most probable case of some columns being treated as float is the absence of value whatsoever. Subsequently these cells had NaN assigned and the whole columns containing NaN's were treated as float.  
  
For now I will merge the dataframes into one and proceed with wrangling.

In [13]:
df = pd.concat([df_2005, df_2006, df_2007], ignore_index=True)

In [14]:
df.shape

(21735733, 29)

Looks good. Now onto more wrangling.

## Assess
### Part II - single data frame

In [15]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21735733 entries, 0 to 21735732
Data columns (total 29 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   Year               21735733 non-null  int64  
 1   Month              21735733 non-null  int64  
 2   DayofMonth         21735733 non-null  int64  
 3   DayOfWeek          21735733 non-null  int64  
 4   DepTime            21319321 non-null  float64
 5   CRSDepTime         21735733 non-null  int64  
 6   ArrTime            21271928 non-null  float64
 7   CRSArrTime         21735733 non-null  int64  
 8   UniqueCarrier      21735733 non-null  object 
 9   FlightNum          21735733 non-null  int64  
 10  TailNum            21735711 non-null  object 
 11  ActualElapsedTime  21271928 non-null  float64
 12  CRSElapsedTime     21734735 non-null  float64
 13  AirTime            21271928 non-null  float64
 14  ArrDelay           21271928 non-null  float64
 15  DepDelay     

In [16]:
# How many unique carriers are there?
len(df.UniqueCarrier.unique())

23

In [17]:
# How many departure and arrival times are missing
df.query('DepTime != DepTime').shape[0]

416412

In [18]:
df.query('ArrTime != ArrTime').shape[0]

463805

In [19]:
# Check on records missing departure or arrival time and if there is other indication of delays within these records
df.query('((DepTime != DepTime) | (ArrTime != ArrTime)) & (CarrierDelay + WeatherDelay + NASDelay + SecurityDelay + LateAircraftDelay == 0)').shape[0]

463805

In [20]:
# Check if flights missing dep or arr time were cancelled
df.query('((DepTime != DepTime) | (ArrTime != ArrTime)) & (CancellationCode == CancellationCode)').shape[0]

416412

In [21]:
# How many flights missing arr time were not cancelled
df.query('(ArrTime != ArrTime) & (CancellationCode != CancellationCode)').shape[0]

47393

In [22]:
# Check for diverted flights
df.query('Diverted == 1').shape[0]

47393

In [23]:
# Just to be sure check if any flights with specified arrival time were diverted (should be 0)
df.query('Diverted == 1 & ArrTime == ArrTime').shape[0]

0

In [24]:
# Check if amount of cancellations corresponds with cancellation code count
df[df['CancellationCode'].notna()].shape[0]

416414

In [25]:
df[df['CancellationCode'].notna()]['Cancelled'].sum()

416412

In [26]:
df[df['CancellationCode'].isna()]['Cancelled'].sum()

0

In [27]:
# Check how many records with arrival delay >= 0 have a different sum of categorized delays
delay_test = df.query('(ArrDelay >= 0) & (ArrDelay != CarrierDelay + WeatherDelay + NASDelay + SecurityDelay + LateAircraftDelay)')
delay_test.shape[0]

4829121

In [28]:
delay_test[['ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']].head(10)

,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
4,3.0,34.0,0,0,0,0,0
9,12.0,-1.0,0,0,0,0,0
15,5.0,19.0,0,0,0,0,0
18,4.0,-4.0,0,0,0,0,0
21,3.0,13.0,0,0,0,0,0
30,4.0,-11.0,0,0,0,0,0
38,4.0,-2.0,0,0,0,0,0
43,14.0,-10.0,0,0,0,0,0
48,14.0,11.0,0,0,0,0,0
77,4.0,30.0,0,0,0,0,0


In [29]:
# It appears that the biggest arrival delay with no recorded cause is 14 minutes.
# Let's check if causes are recorded for 15+ minute delays.
# Outcome of 0 means causes get recorded for 15+ minute delays.
df.query('(ArrDelay >= 15) & (ArrDelay != CarrierDelay + WeatherDelay + NASDelay + SecurityDelay + LateAircraftDelay)').shape[0]

0

In [30]:
# for easier navigation through time-related columns
time_cols = ['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 
             'ArrDelay', 'DepDelay', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

In [31]:
df[time_cols[4:]].describe()

,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,2.127193e+07,2.173474e+07,2.127193e+07,2.127193e+07,2.131932e+07,2.173573e+07,2.173573e+07,2.173573e+07,2.173573e+07,2.173573e+07
mean,1.257125e+02,1.267866e+02,1.023223e+02,8.705460e+00,1.007372e+01,3.427794e+00,7.042450e-01,3.582012e+00,2.481269e-02,4.435478e+00
std,7.110605e+01,7.014748e+01,7.584979e+01,3.681172e+01,3.372605e+01,1.917172e+01,8.841970e+00,1.556168e+01,1.133373e+00,1.962606e+01
min,-6.600000e+01,-1.240000e+03,-1.428000e+03,-9.390000e+02,-1.200000e+03,0.000000e+00,0.000000e+00,-4.900000e+01,0.000000e+00,0.000000e+00
25%,7.500000e+01,7.600000e+01,5.400000e+01,-9.000000e+00,-4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.070000e+02,1.080000e+02,8.400000e+01,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.560000e+02,1.560000e+02,1.310000e+02,1.300000e+01,9.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.879000e+03,1.430000e+03,1.958000e+03,2.598000e+03,2.601000e+03,2.580000e+03,1.510000e+03,1.392000e+03,3.820000e+02,1.366000e+03


In [32]:
# Weirdly, the dataset contains records with negative values for elapsed time and air time, where the smallest possible value is 0.
# How many such cases are there?
df.query('(ActualElapsedTime < 0) | (CRSElapsedTime < 0) | (AirTime < 0)').shape[0]

9873

In [33]:
df.query('(ActualElapsedTime < 0) | (CRSElapsedTime < 0) | (AirTime < 0)')[time_cols].head(10)

,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
204345,23.0,2230,2400.0,2347,-23.0,77.0,-17.0,13.0,113.0,0,0,0,0,0
207898,1710.0,1640,2400.0,1746,410.0,66.0,-15.0,374.0,30.0,0,0,0,0,374
209786,2220.0,2220,2400.0,2348,100.0,88.0,-38.0,12.0,0.0,0,0,0,0,0
216462,2359.0,2220,2400.0,39,-59.0,79.0,-14.0,-39.0,99.0,0,0,0,0,0
221783,1615.0,1555,2400.0,1822,405.0,87.0,-11.0,338.0,20.0,338,0,0,0,0
230160,2400.0,2120,2254.0,2215,-66.0,55.0,-1278.0,39.0,160.0,0,0,0,0,39
231281,2338.0,2220,201.0,39,83.0,79.0,-58.0,82.0,78.0,0,0,0,0,82
324894,2126.0,2104,2411.0,2255,165.0,111.0,-1351.0,76.0,22.0,0,0,54,0,22
324902,2254.0,2104,2455.0,2255,121.0,111.0,-1335.0,120.0,110.0,0,0,120,0,0
324903,2225.0,2104,2420.0,2255,115.0,111.0,-1353.0,85.0,81.0,81,0,4,0,0


The above shows that:
- Usually it's `AirTime` with large negative values that is problematic
- Other time values don't make sense either - they are over 2400 on a 24 h timescale.

In [34]:
# Check records for outright invalid time format
df.query('(DepTime >= 2400) | (CRSDepTime >= 2400) | (ArrTime >= 2400) | (CRSArrTime >= 2400)').shape[0]

21306

### Issues
#### Quality
- Some columns are float64 due to NaN's: `DepTime`, `ArrTime`, `ActualElapsedTime`, `CRSElapsedTime`, `AirTime`, `ArrDelay`, `DepDelay`
- Some time columns are numeric where they could be date/time: `DepTime`, `CRSDepTime`, `ArrTime`, `CRSArrTime`  
Remember about filling missing zeroes up front (eg. 5.0 would be 00:05 hh:mm)
- `UniqueCarrier` could safely be categorical - only 23 different carriers
- In two records `CancellationCode` was provided, but `Cancelled` was set to 0
- Only records with 15 minutes or longer delays have their cause specified - be aware when analyzing
- The dataset contains 0.05% records with negative values for elapsed time and air time
- 0.1% of records contain invalid number for a time, larger or equal to 2400 - equivalent of 24:00 hh:mm
- It seems as though on occassion quality issues related to flight times may appear - eg. flight duration different than the result of arrival time minus departure time.  
Having in mind the sheer size of the dataset and the focus of this project being visualisation rather than wrangling, I will leave the remaining ones untouched at this point in time. May be of interest for deeper analysis.

#### Tidiness
- A number of columns containing repeated non-numeric data should rather have integer indices linked to separate tables: `UniqueCarrier`, `TailNum`, `Origin`, `Dest`, `CancellationCode`. Since extensive cleaning is not the purpose of this project, I will leave it as is.
- `Year`, `Month` and `DayofMonth` could all form a single date column treated as date/time. Will leave as is.

## Clean
#### Make df copy for cleaning

In [35]:
dfc = df.copy()

### The dataset contains 0.05% records with negative values for elapsed time and air time.  
**Columns: `ActualElapsedTime`, `CRSElapsedTime`, `AirTime`.**
#### Define
I will remove these records as they contain clearly invalid data.  
This is only a tiny, insignifficant fraction of the whole dataset, so it will not influence later analysis.

#### Code

In [36]:
dfc = dfc[~(dfc['ActualElapsedTime'] < 0) & ~(dfc['CRSElapsedTime'] < 0) & ~(dfc['AirTime'] < 0)]

#### Test

In [37]:
# Updated df row number
dfc.shape[0]

21725860

In [38]:
# Check if numbers match
assert dfc.shape[0] + df.query('(ActualElapsedTime < 0) | (CRSElapsedTime < 0) | (AirTime < 0)').shape[0] == df.shape[0]

### 0.1% of records contain invalid number for a time, larger or equal to 2400 - equivalent of 24:00 hh:mm
#### Re-assess after above cleaning

In [39]:
# Should be roughly half less than before
max_hour = 2400
dfc.query('(DepTime >= @max_hour) | (CRSDepTime >= @max_hour) | (ArrTime >= @max_hour) | (CRSArrTime >= @max_hour)').shape[0]

11476

In [40]:
# indices list for future re-check
bad_midnight = dfc.query('(DepTime >= @max_hour) | (CRSDepTime >= @max_hour) | (ArrTime >= @max_hour) | (CRSArrTime >= @max_hour)').index

In [41]:
# Check if any of the values exceed 2500
max_hour = 2500
dfc.query('(DepTime >= @max_hour) | (CRSDepTime >= @max_hour) | (ArrTime >= @max_hour) | (CRSArrTime >= @max_hour)').shape[0]

2

In [42]:
dfc.query('(DepTime >= @max_hour) | (CRSDepTime >= @max_hour) | (ArrTime >= @max_hour) | (CRSArrTime >= @max_hour)')[time_cols]

,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
12194387,2203.0,1930,2500.0,2135,177.0,125.0,105.0,205.0,153.0,153,0,52,0,0
12196682,2030.0,1737,2513.0,2049,223.0,132.0,135.0,264.0,173.0,0,173,91,0,0


#### Define
I will manually correct time values for above two records.  
For records with time between 2400 and 2500 I will assume the correctness of the data (in any case it's only a further 0.05% of the original dataset) and set the values to 00mm.
#### Code

In [43]:
# values of > 2500
dfc.loc[dfc.index == 12194387, 'ArrTime'] = 100
dfc.loc[dfc.index == 12196682, 'ArrTime'] = 113
dfc.loc[dfc.index == 12196682, 'ActualElapsedTime'] = 283

In [44]:
# values of 2400 to 2500
max_hour = 2400
check_cols = ['DepTime', 'ArrTime', 'CRSDepTime', 'CRSArrTime']

for col in check_cols :
    dfc.loc[dfc[col] >= max_hour, col] = dfc[col] - 2400

#### Test

In [45]:
# values of > 2500
dfc[dfc.index.isin([12194387, 12196682])][time_cols]

,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
12194387,2203.0,1930,100.0,2135,177.0,125.0,105.0,205.0,153.0,153,0,52,0,0
12196682,2030.0,1737,113.0,2049,283.0,132.0,135.0,264.0,173.0,0,173,91,0,0


In [46]:
# values of 2400 to 2500
dfc[dfc.index.isin(bad_midnight)][time_cols].head()

,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
110,2102.0,2045,0.0,2359,118.0,134.0,102.0,1.0,17.0,0,0,0,0,0
3607,2100.0,1645,0.0,1822,240.0,157.0,126.0,338.0,255.0,0,0,318,0,20
6532,2222.0,2225,0.0,6,98.0,101.0,81.0,-6.0,-3.0,0,0,0,0,0
6561,2203.0,2115,0.0,2317,57.0,62.0,40.0,43.0,48.0,43,0,0,0,0
7120,2127.0,2105,0.0,2352,93.0,107.0,69.0,8.0,22.0,0,0,0,0,0


### Some columns are float64 due to NaN's: `DepTime`, `ArrTime`, `ActualElapsedTime`, `CRSElapsedTime`, `AirTime`, `ArrDelay`, `DepDelay`
### Some time columns are numeric where they could be date/time: `DepTime`, `CRSDepTime`, `ArrTime`, `CRSArrTime`  
#### Define
All flights with NaN values in actual flight time data were either cancelled or diverted - conclusion: no data is missing.  
I will convert hh:mm columns to time and duration columns to Int64 to allow for NaNs.
hh:mm columns will be seen as string/object because time only values cannot be converted to date/time format.

In [47]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21725860 entries, 0 to 21735732
Data columns (total 29 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             int64  
 20  TaxiOut            int64  
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       int64  
 25  WeatherDelay    

#### Code

In [48]:
# Convert not-NaN times to string and format to hh:mm:ss
for col in time_cols[:4] :
    dfc.loc[dfc[col].notna(), col] = dfc[col].apply(lambda x: str(x).split('.')[0]).str.zfill(4).apply(lambda x: x[:2]+':'+x[2:]+':00')

In [49]:
# Convert not-NaN strings to time
for col in time_cols[:4] :
    dfc.loc[dfc[col].notna(), col] = pd.to_datetime(dfc[col], format="%H:%M:%S").dt.time

In [50]:
# Convert appropriate float64 columns with NaNs to Int64
for col in time_cols[4:9] :
    dfc = dfc.astype({col:'Int64'})

#### Test

In [51]:
dfc.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21725860 entries, 0 to 21735732
Data columns (total 29 columns):
 #   Column             Non-Null Count     Dtype 
---  ------             --------------     ----- 
 0   Year               21725860 non-null  int64 
 1   Month              21725860 non-null  int64 
 2   DayofMonth         21725860 non-null  int64 
 3   DayOfWeek          21725860 non-null  int64 
 4   DepTime            21309451 non-null  object
 5   CRSDepTime         21725860 non-null  object
 6   ArrTime            21262093 non-null  object
 7   CRSArrTime         21725860 non-null  object
 8   UniqueCarrier      21725860 non-null  object
 9   FlightNum          21725860 non-null  int64 
 10  TailNum            21725838 non-null  object
 11  ActualElapsedTime  21262093 non-null  Int64 
 12  CRSElapsedTime     21724862 non-null  Int64 
 13  AirTime            21262093 non-null  Int64 
 14  ArrDelay           21262093 non-null  Int64 
 15  DepDelay           21309451 no

In [52]:
dfc[time_cols].head()

,DepTime,CRSDepTime,ArrTime,CRSArrTime,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,16:03:00,16:05:00,17:41:00,17:59:00,158,174,131,-18,-2,0,0,0,0,0
1,15:59:00,16:05:00,17:36:00,17:59:00,157,174,136,-23,-6,0,0,0,0,0
2,16:03:00,16:10:00,17:41:00,18:05:00,158,175,131,-24,-7,0,0,0,0,0
3,15:56:00,16:05:00,17:26:00,17:59:00,150,174,129,-33,-9,0,0,0,0,0
4,19:34:00,19:00:00,22:35:00,22:32:00,121,152,106,3,34,0,0,0,0,0


### `UniqueCarrier` could safely be categorical - only 23 different carriers
#### Define
I will convert `UniqueCarrier` column to category sorted descending by counts.
#### Code

In [53]:
carr_counts = dfc['UniqueCarrier'].value_counts()
carr_counts.index

Index(['WN', 'AA', 'OO', 'DL', 'MQ', 'UA', 'US', 'NW', 'XE', 'CO', 'OH', 'EV',
       'FL', 'YV', 'AS', 'B6', '9E', 'F9', 'HP', 'HA', 'DH', 'AQ', 'TZ'],
      dtype='object')

In [54]:
carr_cat = pd.api.types.CategoricalDtype(carr_counts.index, ordered=True)

In [55]:
dfc['UniqueCarrier'] = dfc['UniqueCarrier'].astype(carr_cat)

#### Test

In [56]:
dfc['UniqueCarrier'].dtype

CategoricalDtype(categories=['WN', 'AA', 'OO', 'DL', 'MQ', 'UA', 'US', 'NW', 'XE', 'CO',
                  'OH', 'EV', 'FL', 'YV', 'AS', 'B6', '9E', 'F9', 'HP', 'HA',
                  'DH', 'AQ', 'TZ'],
, ordered=True)

### In two records `CancellationCode` was provided, but `Cancelled` was set to 0
#### Define
Check manually the reason of inconsistency.
#### Code

In [57]:
dfc[(dfc['CancellationCode'].notna()) & (dfc['Cancelled'] == 0)]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
21681210,2007,12,16,7,15:02:00,14:35:00,16:16:00,15:53:00,B6,74,...,3,29,0,B,0,0,8,0,0,15


In [58]:
# The record has ArrTime in place -> the flight has arrived at destination, hence CancellationCode is not needed
dfc.loc[dfc.index == 21681210, 'CancellationCode'] = np.nan

#### Test

In [59]:
dfc[dfc.index == 21681210]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
21681210,2007,12,16,7,15:02:00,14:35:00,16:16:00,15:53:00,B6,74,...,3,29,0,NaN,0,0,8,0,0,15


## Relations potentially worth exploring
The dataset is now of better quality.  
Below I will consider which features are worth exploring.  
In the end I will strip and split the data so that only useful part remains.  
- Actual vs Planned flight time
- Distance vs Arrival Delay
- Arrival Delay vs Departure Delay
- Arrival Delay vs Unique Carrier
- Delay by category by year by month or even day/day of week
- Delays by Origin and Destination
- Diverting by Origin and Destination
- Cancellation by category

## Output
Eport data wrangled so far for possible future exploration.  
Prepare smaller chunks of data for analyses.

#### Export clean data to csv

In [60]:
dfc.to_csv('./data/flights_clean.csv', index=False)

#### Get rid of columns that won't be used at all
Namely: `FlightNum`, `TailNum`, `AirTime`, `TaxiIn`, `TaxiOut`.

In [61]:
dfc.drop(columns=['FlightNum', 'TailNum', 'AirTime', 'TaxiIn', 'TaxiOut'], inplace=True)

#### Have separate dataframes / files: one for diverted or cancelled flights and the second for all other flights
This will allow for dropping NaN columns from each set and ease up on future analysis.

In [62]:
df_cancelled = dfc[(dfc['Cancelled'] == 1) | (dfc['Diverted'] == 1)].copy()

In [63]:
df_cancelled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463767 entries, 7 to 21735717
Data columns (total 24 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   Year               463767 non-null  int64   
 1   Month              463767 non-null  int64   
 2   DayofMonth         463767 non-null  int64   
 3   DayOfWeek          463767 non-null  int64   
 4   DepTime            47358 non-null   object  
 5   CRSDepTime         463767 non-null  object  
 6   ArrTime            0 non-null       object  
 7   CRSArrTime         463767 non-null  object  
 8   UniqueCarrier      463767 non-null  category
 9   ActualElapsedTime  0 non-null       Int64   
 10  CRSElapsedTime     462769 non-null  Int64   
 11  ArrDelay           0 non-null       Int64   
 12  DepDelay           47358 non-null   Int64   
 13  Origin             463767 non-null  object  
 14  Dest               463767 non-null  object  
 15  Distance           463767 non-nu

In [64]:
# Dropping wholly NaN columns
df_cancelled.drop(columns=['ArrTime', 'ActualElapsedTime', 'ArrDelay'], inplace=True)

In [65]:
df_normal = dfc[(dfc['Cancelled'] == 0) & (dfc['Diverted'] == 0)]

In [66]:
df_normal.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21262093 entries, 0 to 21735732
Data columns (total 24 columns):
 #   Column             Non-Null Count     Dtype   
---  ------             --------------     -----   
 0   Year               21262093 non-null  int64   
 1   Month              21262093 non-null  int64   
 2   DayofMonth         21262093 non-null  int64   
 3   DayOfWeek          21262093 non-null  int64   
 4   DepTime            21262093 non-null  object  
 5   CRSDepTime         21262093 non-null  object  
 6   ArrTime            21262093 non-null  object  
 7   CRSArrTime         21262093 non-null  object  
 8   UniqueCarrier      21262093 non-null  category
 9   ActualElapsedTime  21262093 non-null  Int64   
 10  CRSElapsedTime     21262093 non-null  Int64   
 11  ArrDelay           21262093 non-null  Int64   
 12  DepDelay           21262093 non-null  Int64   
 13  Origin             21262093 non-null  object  
 14  Dest               21262093 non-null  object  
 

In [67]:
df_normal = df_normal.drop(columns=['Cancelled', 'CancellationCode', 'Diverted'])

In [68]:
df_normal.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21262093 entries, 0 to 21735732
Data columns (total 21 columns):
 #   Column             Non-Null Count     Dtype   
---  ------             --------------     -----   
 0   Year               21262093 non-null  int64   
 1   Month              21262093 non-null  int64   
 2   DayofMonth         21262093 non-null  int64   
 3   DayOfWeek          21262093 non-null  int64   
 4   DepTime            21262093 non-null  object  
 5   CRSDepTime         21262093 non-null  object  
 6   ArrTime            21262093 non-null  object  
 7   CRSArrTime         21262093 non-null  object  
 8   UniqueCarrier      21262093 non-null  category
 9   ActualElapsedTime  21262093 non-null  Int64   
 10  CRSElapsedTime     21262093 non-null  Int64   
 11  ArrDelay           21262093 non-null  Int64   
 12  DepDelay           21262093 non-null  Int64   
 13  Origin             21262093 non-null  object  
 14  Dest               21262093 non-null  object  
 

#### Export split data to csv's

In [69]:
df_cancelled.to_csv('./data/flights_cancelled.csv', index=False)
df_normal.to_csv('./data/flights_normal.csv', index=False)